In [1]:
import torch
import numpy as np
import torch.nn as nn

if torch.cuda.is_available(): 
 dev = "cuda:0" 
else: 
 dev = "cpu" 
device = torch.device(dev) 

In [5]:
features = torch.load('batched_data\\features').to(device)
labels = torch.load('batched_data\\labels').to(device)

In [6]:
print(features.shape)
print(labels.shape)

torch.Size([6977, 17280, 2])
torch.Size([6977, 17280])


In [7]:
class model(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(model, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=1, 
                            bias=True, batch_first=True, dropout=0.0, 
                            bidirectional=False, proj_size=0, device=device, 
                            dtype=None)
        
        self.linear = nn.Linear(hidden_size, 1)
        self.softMax = nn.LogSoftmax(dim=1)

        self.series_length = 17280

    def forward(self, x):
        x, _ = self.lstm(x)

        x = self.stack(x)
        x = self.linear(x)
        x = self.unstack(x)

        x = self.softMax(x)
        return x

    def stack(self, x):
        x = x.reshape(x.shape[0] * self.series_length, x.shape[2])
        return x
    
    def unstack(self, x):
        x = x.reshape(x.shape[0] // self.series_length, self.series_length)
        print(x.shape)
        return x

In [8]:
from torch import optim

inputSize = 2
hiddenSize = 20

lstm = model(inputSize, hiddenSize).to(device)
lossFunction = nn.MSELoss()
optimizer = optim.Adam(lstm.parameters(), lr = 1e-3)


In [10]:
from matplotlib import pyplot as plt

batchSize = 200
numIters = 100
epochs = 10

for i in range(0, epochs * batchSize, batchSize):
    runningLoss = 0.0

    batch = features[i:i+batchSize].to(device)
    batchedLabels = labels[i:i+batchSize].double().to(device)

    for _ in range(numIters):
        out = lstm(batch).squeeze()
    
        loss = lossFunction(out, batchedLabels)
        
        optimizer.zero_grad()

        loss.backward()
        optimizer.step()

        runningLoss += loss.detach().cpu().numpy()

    print("epoch: ", i/batchSize, " loss: ", runningLoss)
    plt.scatter(i, runningLoss, color="b")


c:\Users\yotam\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([200, 17280])) that is different to the input size (torch.Size([3456000])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (3456000) must match the size of tensor b (17280) at non-singleton dimension 1

In [7]:
class model(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(model, self).__init__()
        self.encoder = nn.TransformerEncoderLayer(input_size, nhead=2, batch_first=True, dim_feedforward=hidden_size)
        self.softMax = nn.LogSoftmax(dim=-1)

    def forward(self, x):
        x, _ = self.encoder(x)
        x = self.softMax(x)
        return x
